In [1]:
import pandas as pd
import numpy as np
import multiprocessing
import contractions
import os
import text_preprocessing as tp

from nltk.corpus import stopwords
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
from scipy.special import softmax
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor
from functools import partial
import time

/home/vscode/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
filename = 'reviews'
source_review_file_diretory_path = "./dataset/"
split_dataset_directory_path = './dataset/split_dataset/'    # contains the split files
done_directory_path = './dataset/split_dataset/done/' # contains split files that were processed
sentiment_directory_path = './dataset/split_dataset/sentiment/' # contains split files that were processed

In [3]:
# def split_and_save_csv(input_file, output_prefix, chunk_size):
#     # Read the CSV file
#     df = pd.read_csv(input_file)

#     # Split the DataFrame into chunks
#     chunks = [df[i:i + chunk_size] for i in range(0, len(df), chunk_size)]

#     # Save each chunk to a separate CSV file
#     for i, chunk in enumerate(chunks):
#         output_file = f"{output_prefix}_{i + 1}.csv"
#         chunk.to_csv(output_file, index=False)
#         print(f"Saved {len(chunk)} records to {output_file}")


# if not os.path.exists(split_dataset_directory_path):
#     os.makedirs(split_dataset_directory_path)

# input_csv = source_review_file_diretory_path + filename + ".csv"  # Replace with your actual CSV file
# output_prefix = split_dataset_directory_path + filename      # Prefix for output files
# chunk_size = 100                     # Number of records per chunk

# split_and_save_csv(input_csv, output_prefix, chunk_size)


In [4]:
MODEL = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)

# Initialize the English stop words list
list_of_stopwords = set(stopwords.words('english'))


def data_preprocessing(text):
    text = tp.remove_email_address(text)
    text = tp.remove_hyperlink(text)
    text = tp.replace_whitespace(text)
    text = tp.remove_stopwords(text, list_of_stopwords)
    return text


def polarity_score_roberta(data):
    # Do basic data pre-processing
    data_preprocessing(data)

    # Specify the maximum sequence length
    max_length = 512  # Adjust this based on the model's maximum sequence length

    # Tokenize and truncate/pad the input text
    encoded_text = tokenizer(data, return_tensors='tf', max_length=max_length, truncation=True, padding=True)
    
    output = model(**encoded_text)
    scores = output[0][0].numpy()
    scores = softmax(scores)
    
    scores_dict = {
        "roberta_neg": scores[0],
        'roberta_neu': scores[1],
        'roberta_pos': scores[2]
    }
    
    return scores_dict

# Define the function to generate labels
def generate_roberta_labels(data, positive_threshold=0.5, negative_threshold=0.5):
    # Get roberta scores
    scores = polarity_score_roberta(data)
    roberta_neg, roberta_neu, roberta_pos = scores['roberta_neg'], scores['roberta_neu'], scores['roberta_pos']

    sentiment_results_dict = {'positive': roberta_pos, 
                              'negative': roberta_neg, 
                              'neutral': roberta_neu}

    highest_sentiment = max(sentiment_results_dict.items(), key=lambda x: x[1])

    if highest_sentiment[0] == 'positive':    
        if highest_sentiment[1] >= 0.8:        
            return "Strongly Positive" 
        else:        
            return "Positive"
    elif highest_sentiment[0] == 'negative':    
        if highest_sentiment[1] >= 0.8:        
            return "Strongly Negative"
        else:
            return "Negative"
    else:
        return "Neutral"


def process_row(df):
    df['roberta_sentiment'] = df['review_detail'].apply(lambda x : generate_roberta_labels(x))
    return df

# NOTE: Change the number of threads depending on device's CPU core
def parallel_processing(df, func, num_threads=3):
    # Split the DataFrame into chunks for parallel processing
    chunks = np.array_split(df, num_threads)

    # Use ProcessPoolExecutor for parallel processing
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        # Map the function to each chunk of the DataFrame in parallel
        results = list(executor.map(func, chunks))

    # Concatenate the results
    result_df = pd.concat(results, ignore_index=True)
    return result_df


def list_files(directory):
    files = []
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath):
            files.append(filepath)
    return files
   

if __name__ == "__main__":

    files_in_directory = list_files(done_directory_path)

    # Create 'done' directory inside the 'split_dataset
    if not os.path.exists(done_directory_path):
        print("Creating 'done' directory...")
        os.makedirs(done_directory_path)

    # Create 'sentiment' directory inside the 'split_dataset
    if not os.path.exists(sentiment_directory_path):
        print("Creating 'sentiment' directory...")
        os.makedirs(sentiment_directory_path)

    for input_filepath in files_in_directory:

        try:
            base_filename = os.path.basename(input_filepath)
            no_ext_base_filename = os.path.splitext(base_filename)[0]
            output_filepath = sentiment_directory_path + no_ext_base_filename + "_sentiment.csv"


            print(f"Reading source the file: {input_filepath}")
            if not os.path.exists(output_filepath):
                start_time = time.time()
                movie_reviews_df = pd.read_csv(input_filepath)
                
                print("Parallel processing of movie reviews...")
                movie_reviews_df = parallel_processing(movie_reviews_df, process_row)
                end_time = time.time()
                elapsed_time = end_time - start_time

                print(f"Execution time: {elapsed_time} seconds")
                
                # Save to csv file
                movie_reviews_df.to_csv(output_filepath, index=False)

                print(f"\nSuccessfully saved the file with sentiments in {output_filepath}")

                # Move the file that was processed to the done folder
                destination_file = os.path.join(done_directory_path, base_filename)
                os.rename(input_filepath, destination_file)


                print(f"\nMove source file to done folder: {destination_file}")
            else:
                print(f"\nFile already processed! Moving ahead!!")
        except Exception as err:
            print(f"ERROR: {err}")
            print(f"File: {input_filepath}")

2023-11-13 12:02:35.193950: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-13 12:02:36.215740: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-13 12:02:36.219409: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-13 12:02:37.861417: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the chec

Reading source the file: ./dataset/split_dataset/done/reviews_1.csv

File already processed! Moving ahead!!
Reading source the file: ./dataset/split_dataset/done/reviews_10.csv

File already processed! Moving ahead!!
Reading source the file: ./dataset/split_dataset/done/reviews_100.csv

File already processed! Moving ahead!!
Reading source the file: ./dataset/split_dataset/done/reviews_101.csv

File already processed! Moving ahead!!
Reading source the file: ./dataset/split_dataset/done/reviews_102.csv

File already processed! Moving ahead!!
Reading source the file: ./dataset/split_dataset/done/reviews_103.csv

File already processed! Moving ahead!!
Reading source the file: ./dataset/split_dataset/done/reviews_104.csv

File already processed! Moving ahead!!
Reading source the file: ./dataset/split_dataset/done/reviews_105.csv

File already processed! Moving ahead!!
Reading source the file: ./dataset/split_dataset/done/reviews_106.csv

File already processed! Moving ahead!!
Reading sourc